# Renaissance Trading Bot — ML Model Training (Colab)

GPU-accelerated training for the 3 ML models:
- **Quantum Transformer** (83→288→4 blocks→1)
- **Bidirectional LSTM** (83→292 bidir→1)
- **Dilated CNN** (83 channels→5 blocks→332→1)

Uses real Coinbase OHLCV data and the production feature pipeline.

In [ ]:
# Cell 1: Install dependencies
!pip install torch numpy pandas

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
if device == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# Cell 2: Upload project files
# Option A: Mount Google Drive (if project is in Drive)
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/bitcoin-trading-bot-renaissance

# Option B: Upload a zip of the project
# from google.colab import files
# uploaded = files.upload()  # Upload project.zip
# !unzip project.zip -d /content/project
# %cd /content/project

# Option C: Clone from git (if available)
# !git clone <your-repo-url> /content/project
# %cd /content/project

import sys, os
PROJECT_ROOT = os.getcwd()
sys.path.insert(0, PROJECT_ROOT)
print(f'Project root: {PROJECT_ROOT}')
print(f'Files: {os.listdir(".")[:20]}')

In [ ]:
# Cell 3: Fetch training data from Coinbase API
from scripts.training.fetch_training_data import download_all

PAIRS = ['BTC-USD', 'ETH-USD', 'SOL-USD', 'DOGE-USD', 'AVAX-USD', 'LINK-USD']
DAYS = 30

pair_dfs = download_all(PAIRS, DAYS)

print(f'\nDownloaded data:')
for pair, df in pair_dfs.items():
    print(f'  {pair}: {len(df)} candles ({len(df)*5/60/24:.1f} days)')

In [ ]:
# Cell 4: Train all models (GPU-accelerated)
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

from scripts.training.train_all import train_all

EPOCHS = 100  # Reduce for quick test: EPOCHS = 5

results = train_all(days=DAYS, epochs=EPOCHS, pairs=PAIRS)

In [ ]:
# Cell 5: Display results
import json

print('Training Results')
print('=' * 60)
for model_name, r in results.items():
    print(f'\n{model_name}:')
    for k, v in r.items():
        if isinstance(v, float):
            print(f'  {k}: {v:.4f}')
        else:
            print(f'  {k}: {v}')

# Check model files
import os
models_dir = 'models/trained'
print(f'\nModel files in {models_dir}:')
for f in sorted(os.listdir(models_dir)):
    if f.endswith('.pth'):
        size_kb = os.path.getsize(os.path.join(models_dir, f)) / 1024
        print(f'  {f}: {size_kb:.1f} KB')

# Verify models load correctly
from ml_model_loader import load_trained_models
models = load_trained_models()
print(f'\nLoaded models: {list(models.keys())}')

In [ ]:
# Cell 6: Download trained .pth files
# Option A: Download directly (Colab)
# from google.colab import files
# for f in os.listdir('models/trained'):
#     if f.endswith('.pth'):
#         files.download(os.path.join('models/trained', f))

# Option B: Copy to Google Drive
# import shutil
# drive_dest = '/content/drive/MyDrive/trained_models/'
# os.makedirs(drive_dest, exist_ok=True)
# for f in os.listdir('models/trained'):
#     if f.endswith('.pth'):
#         shutil.copy2(os.path.join('models/trained', f), drive_dest)
#         print(f'Copied {f} to Drive')

print('Uncomment the download method above to save your trained models.')
print('\nModel files:')
for f in sorted(os.listdir('models/trained')):
    if f.endswith('.pth'):
        print(f'  {f}')